In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [9]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware
from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model = "qwen/qwen3-next-80b-a3b-instruct")
mini_model = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=mini_model,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [10]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\n## SESSION INTENT\nThe user's primary goal is to gather information about the fictional city of Lunapolis, specifically its weather and population.\n\n## SUMMARY\nThe conversation history reveals the following important context:\n- Lunapolis is the capital of the moon.\n- The weather in Lunapolis is extreme, with clear skies, a high of 120C, and a low of -100C.\n- There are 100,000 cheese miners living in Lunapolis.\n- The cheese miners' union is unhappy with the new president and is likely to strike.\n\n## ARTIFACTS\nNone\n\n## NEXT STEPS\nTo achieve the session intent, the next steps are to:\n- Provide more information about the cheese miners' union and their grievances.\n- Offer potential solutions to resolve the strike and improve the living conditions for the cheese miners.\n- Explore the implications of the strike on the moon's economy and Lunapolis' infrastructure.", additional_kwargs={'lc_sour

In [11]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

## SESSION INTENT
The user's primary goal is to gather information about the fictional city of Lunapolis, specifically its weather and population.

## SUMMARY
The conversation history reveals the following important context:
- Lunapolis is the capital of the moon.
- The weather in Lunapolis is extreme, with clear skies, a high of 120C, and a low of -100C.
- There are 100,000 cheese miners living in Lunapolis.
- The cheese miners' union is unhappy with the new president and is likely to strike.

## ARTIFACTS
None

## NEXT STEPS
To achieve the session intent, the next steps are to:
- Provide more information about the cheese miners' union and their grievances.
- Offer potential solutions to resolve the strike and improve the living conditions for the cheese miners.
- Explore the implications of the strike on the moon's economy and Lunapolis' infrastructure.


## Trim/delete messages

In [13]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [14]:
agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [15]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='47f70713-7dab-4d06-998f-fdfa5c42a1ec'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='08a63ef2-5db5-4dab-bd69-675b349d2d98', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='d513a9fb-8e85-4535-905f-70851d4ef215'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='bf6c3135-31a9-4b54-92e3-ad7d43f67db3', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='88e2b59f-91cb-4f0c-9dbd-44090d805abe'),
              AIMessage(content='I can’t physically check the temperature of your device — but 

In [16]:
print(response["messages"][-1].content)

I can’t physically check the temperature of your device — but here’s what you can do:

### 🔍 Check the Temperature Yourself:
1. **Turn off the device** (if it’s on) and unplug it.
2. **Wait 5–10 minutes** for it to cool down (if it was running).
3. **Gently touch the casing** — especially near vents, the power supply, or battery area.
   - **Warm?** → Normal during operation.
   - **Hot to the touch?** → Could indicate overheating or internal failure.
   - **Cold?** → May mean no power is reaching internal components.

### 🚨 If It’s Overheated:
- Let it cool completely (at least 30 minutes).
- Clean dust from vents with compressed air.
- Try turning it on again.

### ❄️ If It’s Cold (and plugged in):
- The power isn’t reaching the internal circuitry.
- Possible causes:
  - Faulty power cable or adapter
  - Blown fuse or damaged power supply
  - Failed motherboard or battery

### ✅ Next Steps:
1. **Try a different power outlet** or known-good power cable.
2. **If it’s a laptop:** Remove